In [ ]:
#!pip install -q torch==1.4.0 torchvision==0.5

In [ ]:
!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install torch-scatter==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-sparse==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-cluster==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-spline-conv==latest+cu101 -f https://pytorch-geometric.com/whl/torch-1.5.0.html
!pip install torch-geometric

In [3]:
cd drive/My\ Drive

/content/drive/My Drive


In [4]:

from torch.autograd import Variable
import torch_geometric.transforms
from torch_geometric.transforms import RadiusGraph
from torch_geometric.transforms import knn_graph
import torch_geometric.data
import torch 
from torch_geometric.data import Data
import numpy as np
import pandas as pd
from scipy.spatial import distance_matrix
import torch
import pyarrow as pa
import pyarrow.parquet as pq
import h5py
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from torch_geometric.nn import knn_graph

parquet = pq.ParquetFile('./Boosted_Jets_Sample-1.snappy.parquet')
cols = None

def jets2(number1,number2):
    allCords=[]
    graphs=[]
    allFeats2=[]
    grIndex=[]
    for i in range(number1,number2):

        data = parquet.read_row_group(i, columns=cols).to_pydict() ## Select parquet file to open
        
        data['X_jets'] = np.float32(data['X_jets']) [0] 
        
        ecal=data['X_jets'][1]  ## Select ECAL Data out of the 3 available channels

        ecal[ecal<=1e-3]=0  ## Remove noisy values
        
        xhit2,yhit2=np.nonzero(ecal)  ## Select hits in detector

        eneEcal=ecal[xhit2,yhit2]  ## Select energies of hits
   
        feats=np.transpose(np.vstack((xhit2,yhit2,eneEcal)))  ## concatenate x,y locations and energies (3 features in total)
        
        cords=feats[:,[0,1]] ## cords = x,y coordinates 
        
        allFeats=torch.from_numpy(feats).float()  ## features to tensors 
        
        cords2=torch.from_numpy(cords)  ## coordinates to tensors 
       
        edge_index = knn_graph(cords2, k=4, batch=None, loop=True)  ## Create knn graph adjacency matrix 
        data=Data(x=allFeats,edge_index=edge_index) ## Create graph data with feature matrix x and adjacency matrix edge_index
        
        graphs.append(data)


    return graphs


In [ ]:
testing1=jets2(0,30000)

In [ ]:
testing2=jets2(0,30000)

In [ ]:
testingComb=testing1+testing2

In [ ]:
torch.save(testingComb,'ECAL_f01_june.pt')